# CORD 19 data

This file goes through the data used for this project.

In [1]:
import wget
import pandas as pd
import numpy as np
import seaborn as sns
import json
import math

Dowloading the data. Relevancy data and topic data.

These files can be explained here: https://ir.nist.gov/covidSubmit/data.html

In [2]:
#Topics; total of 50 topics
    # Query: 
    # Question:
    # Nerrative: 
url_xml = "https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml"
filename_xml = wget.download(url_xml)


#Relevance data:
    # topic_id: corresponds to the topic [1-50]
    # Round_id: What round was this query added, 1,2...,5,
    #           whith round 0.5, the documents were selected from runs produced by the organizers that were not part of official runs. 
    # cord_uid: the cord uid for the document
    # relevancy: 0 - not relevant
    #            1 - partially relevant
    #            2 - fully relevant
url_txt = "https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt"
filname_txt = wget.download(url_txt)


100% [..........................................................................] 1142244 / 1142244

In [3]:
import xml.etree.ElementTree as ET

topics = {}
root = ET.parse("topics-rnd5.xml").getroot()
for topic in root.findall("topic"):
    topic_number = topic.attrib["number"]
    topics[topic_number] = {}
    for query in topic.findall("query"):
        topics[topic_number]["query"] = query.text
    for question in topic.findall("question"):
        topics[topic_number]["question"] = question.text
    for narrative in topic.findall("narrative"):
        topics[topic_number]["narrative"] = narrative.text

In [4]:
import pandas as pd

relevance_data = pd.read_csv("qrels-covid_d5_j0.5-5.txt", sep=" ", header=None)
relevance_data.columns = ["topic_id", "round_id", "cord_uid", "relevancy"]

In [5]:
print(len(topics))
topics["5"]

50


{'query': 'animal models of COVID-19',
 'question': 'what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?',
 'narrative': 'Papers that describe the results  of testing drugs that bind to spike proteins of the virus or any other drugs in any animal models. Papers about SARS-CoV-2 infection in cell culture assays are also relevant.'}

In [6]:
n = len(topics)
query = [0 for _ in range(n)]
question = [0 for _ in range(n)]
narrative = [0 for _ in range(n)]

for i , topic in enumerate(topics):
    row = topics[topic]
    query[i] = row["query"]
    question[i] = row["question"]
    narrative[i] = row["narrative"]

topics_df = pd.DataFrame({
    "query" : query,
    "question" : question,
    "narrative" : narrative},
    index = [i for i in range(1,len(query)+1)])

topics_df.head()

,query,question,narrative
1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


Save dataframe

In [7]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\"
topics_df.to_csv(path + "topics.csv")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\topics.csv'

In [8]:
relevance_data.head()

,topic_id,round_id,cord_uid,relevancy
0,1,4.5,005b2j4b,2
1,1,4.0,00fmeepz,1
2,1,0.5,010vptx3,2
3,1,2.5,0194oljo,1
4,1,4.0,021q9884,1


Save dataframe

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\"
relevance_data.to_csv(path + "relevance_data.csv")

In [ ]:
relevance_data_topic5 = relevance_data[relevance_data["topic_id"] == 5]

In [ ]:
relevance_data_topic5.head()

In [ ]:
# 02f0opkr; 0, 02n30zc5; 2, 02q9y011; 1

In [13]:
relevance_data[relevance_data["relevancy"] == -1].head()

,topic_id,round_id,cord_uid,relevancy
55873,38,5.0,9hbib8b3,-1
69173,50,5.0,ucipq8uk,-1


In [9]:
import plotly.express as px

fig = px.histogram(relevance_data, x="topic_id", color = "relevancy")
fig.show()